# Montevideo data tranformation to GTFS

First of all we should the data from their website.

The file we'll use to create stop_times.txt can be found here: http://www.montevideo.gub.uy/sites/default/files/datos/uptu_pasada_variante.zip

The file we'll use to create stops.txt can be found here: http://intgis.montevideo.gub.uy/sit/php/common/datos/generar_zip2.php?nom_tab=v_uptu_paradas&tipo=gis

The file we'll use to create routes.txt can be found here: http://intgis.montevideo.gub.uy/sit/php/common/datos/generar_zip2.php?nom_tab=v_uptu_lsv_destinos&tipo=gis

In [654]:
# Import the libraries needed

import pandas as pd
import geopandas as gpd
import numpy as np

In [655]:
# Replace the paths for the path to the files in your computer

url_stop_times = "/Users/santiagotoso/GoogleDrive/Master/R/Remix/Montevideo/uptu_pasada_variante.csv"
s_path = "/Users/santiagotoso/GoogleDrive/Master/R/Remix/Montevideo/stops/v_uptu_paradas.shp"
rt_path = "/Users/santiagotoso/GoogleDrive/Master/R/Remix/Montevideo/v_uptu_lsv_destinos/v_uptu_lsv_destinos.shp"

In [657]:
# Import the data as data frames and geo data frames

st = pd.read_csv(url_stop_times, sep=';')
s = gpd.read_file(s_path)
rt = gpd.read_file(rt_path)

In [658]:
# We check the projection of the file. 
# If it doesn't have any you can always drag and drop the file to QGIS 
# go to "Properties" of the layer and get the projection from there.

s.crs
rt.crs

{}

In [659]:
# In this case it didn't have projection information so we
# looked for it in QGIS
# We set the right projection before changing it to the one we need

s.crs = {'init': 'epsg:32721'}
rt.crs = {'init': 'epsg:32721'}

In [660]:
# We reproject to the system we need to create the GTFS

strans = s.to_crs({'init': 'epsg:4326'})
rtrans = rt.to_crs({'init': 'epsg:4326'})

# We check our projection is right now

strans.crs

{'init': 'epsg:4326'}

## Agency.txt

In [661]:
# Create the data frame

d = {'agency_id': [1], 
     'agency_name': ["STM Sistema de Transporte Metropolitano"], 
     'agency_url':["http://www.montevideo.gub.uy/transito-y-transporte/stm-sistema-de-transporte-metropolitano"],
     'agency_timezone':["America/Montevideo"]}
agency = pd.DataFrame(data = d, index = [''])
agency

,agency_id,agency_name,agency_url,agency_timezone
,1,STM Sistema de Transporte Metropolitano,http://www.montevideo.gub.uy/transito-y-transp...,America/Montevideo


In [662]:
# Save the data frame
# Change the path to your output desired address

agency.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/agency.txt", index = False)

## Calendar.txt

In [663]:
# Create the data frame

d = {'service_id': [1,2,3], 
     'monday':[1,0,0],
     'tuesday': [1,0,0],
     'wednesday':[1,0,0],
     'thursday':[1,0,0],
     'friday':[1,0,0],
     'saturday':[0,1,0],
     'sunday':[0,0,1],
     'start_date':['20190101', '20190101', '20190101'],
     'end_date':['20191231', '20191231', '20191231']
    }

calendar = pd.DataFrame(data = d)
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,1,1,1,1,1,0,0,20190101,20191231
1,2,0,0,0,0,0,1,0,20190101,20191231
2,3,0,0,0,0,0,0,1,20190101,20191231


In [664]:
# Save the data frame
# Change the path to your output desired address

calendar.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/calendar.txt", index = False)

## Stops.txt

In [665]:
# First we search for NA values and save them adding a '-'
# This will helps us when we create the data frame

boolean_array = strans['ESQUINA'].isna()
strans.loc[boolean_array, ['ESQUINA']] = ' '
strans[['CALLE','ESQUINA']].isna().any()

CALLE      False
ESQUINA    False
dtype: bool

In [666]:
# Create the data frame

stops = pd.DataFrame(columns = ['stop_id', 'stop_name', 'stop_lat', 'stop_lon'])
stops['stop_id'] = strans["COD_UBIC_P"].astype('float').astype('int').astype('str').unique()
stops['stop_name'] = 'calle ' + strans.CALLE + " - esquina " + strans.ESQUINA + ' - id ' + stops.stop_id
stops['stop_name'] = stops.stop_name.unique()
stops['stop_lat'] = strans.geometry.y.unique()
stops['stop_lon'] = strans.geometry.x.unique()
len(stops)

4712

In [667]:
# Save the data frame
# Change the path to your output desired address

stops.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/stops.txt", index = False)

# Routes.txt

In [668]:
# Create the fields we need for the GTFS from their data

route_id = rtrans.COD_SUBLIN.astype('str') + '-' + rtrans.DESC_SUBLI.astype('str')
route_id = route_id.unique()
route_short_name = rtrans.COD_LINEA.astype('str') + '-' + rtrans.COD_SUBLIN.astype('str') + '-' + rtrans.DESC_SUBLI.astype('str')
route_short_name = route_short_name.unique()
route_short_name = pd.DataFrame(data = route_short_name)

In [669]:
# Create the data frame

routes = pd.DataFrame(data = route_id)
routes.columns = ['route_id']
routes['agency_id'] = '1'
routes['route_short_name'] = route_short_name
routes['route_long_name'] = ''
routes['route_type'] = '3'
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,1-CIUDAD VIEJA - MALVIN,1,1-1-CIUDAD VIEJA - MALVIN,,3
1,2-CJO. J. DE AMÃRICA - PCIO. DE LA LUZ,1,2-2-CJO. J. DE AMÃRICA - PCIO. DE LA LUZ,,3
2,3-PEÃAROL - PARQUE RODÃ,1,3-3-PEÃAROL - PARQUE RODÃ,,3
3,246-GRUTA DE LOURDES - PARQUE RODO,1,3-246-GRUTA DE LOURDES - PARQUE RODO,,3
4,4-PORTONES - PLAZA ESPAÃA,1,4-4-PORTONES - PLAZA ESPAÃA,,3


In [670]:
# Save the data frame
# Change the path to your output desired address

routes.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/routes.txt", index = False)

# Trips.txt

In st some rows are from the same trip but have different `tipo_dia`.
This is because `tipo_dia` changes when you go beyond 24:00h.
When this happens the column `dia_anterior` has the value `S`.
In this case this breaks our pattern for calculating trip_id and we'll
need to change tipo_dia for those rows with `dia_anterior = S`.
Another problem is that `dia_anterior = *` for trips that take place after a Sunday at 24:00, but gets the `tipo_dia = 3` anyway.
This makes our formula for the `trip_id` not good cause two trips could
have the same `id` even they are not the same.
Looks like the first thing to do is to correct the times (`hora` and `frecuencia`)
and `tipo_dia`.

In [671]:
# We start by joining the data frame with the stop times and the data frame with the routes
# Take into account that we'll consider the each combination "COD_SUBLIN"-"DESC_SUBLI" as 
# a route_id and each pattern inside of those will be the patterns.
# The only unique fields are the one indicating the patterns: "cod_variante" and "COD_VARIAN"
# respectively.

rtrans['route_id'] = rtrans.COD_SUBLIN.astype('str') + '-' + rtrans.DESC_SUBLI.astype('str')
st1 = pd.merge(st, rtrans, how='left', left_on=['cod_variante'], right_on=['COD_VARIAN'])
st1.head()

,tipo_dia,cod_variante,frecuencia,cod_ubic_parada,ordinal,hora,dia_anterior,GID,COD_LINEA,DESC_LINEA,...,DESC_SUBLI,COD_VARIAN,DESC_VARIA,COD_VAR_01,COD_ORIGEN,DESC_ORIGE,COD_DESTIN,DESC_DESTI,geometry,route_id
0,1,7973,22520,2078,1,2253,N,46722127.0,81.0,173,...,PORTONES - NUEVO CENTRO,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO
1,1,7973,22520,2079,2,2253,N,46722127.0,81.0,173,...,PORTONES - NUEVO CENTRO,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO
2,1,7973,22520,2340,3,2255,N,46722127.0,81.0,173,...,PORTONES - NUEVO CENTRO,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO
3,2,7973,22480,2078,1,2249,N,46722127.0,81.0,173,...,PORTONES - NUEVO CENTRO,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO
4,2,7973,22480,2079,2,2249,N,46722127.0,81.0,173,...,PORTONES - NUEVO CENTRO,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO


In [672]:
# First we have to create the column "arrival_time" with the format needed for GTFS.
# This will allow us to solve the problem with "dia_anterio" and "tipo_dia" described
# previously.
# We start by creating the function that'll do this

def arrival_time(hora):
    'Creates the arrival_time field following the GTFS standard.'
    if len(str(hora)) == 1:
        return '00:0' + str(hora) + ':00'
    elif len(str(hora)) == 2:
        return '00:' + str(hora) + ':00'
    elif len(str(hora)) == 3:
        return '0' + str(hora)[0] + ':' + str(hora)[-2:] + ':00'
    elif len(str(hora)) == 4:
        return str(hora)[:-2] + ':' + str(hora)[-2:] +  ':00'
    else:
        return 'Not Known'

In [673]:
# Now we can create the column

st1['arrival_time'] = st1['hora'].apply(arrival_time)
st1.head()

,tipo_dia,cod_variante,frecuencia,cod_ubic_parada,ordinal,hora,dia_anterior,GID,COD_LINEA,DESC_LINEA,...,COD_VARIAN,DESC_VARIA,COD_VAR_01,COD_ORIGEN,DESC_ORIGE,COD_DESTIN,DESC_DESTI,geometry,route_id,arrival_time
0,1,7973,22520,2078,1,2253,N,46722127.0,81.0,173,...,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:53:00
1,1,7973,22520,2079,2,2253,N,46722127.0,81.0,173,...,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:53:00
2,1,7973,22520,2340,3,2255,N,46722127.0,81.0,173,...,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:55:00
3,2,7973,22480,2078,1,2249,N,46722127.0,81.0,173,...,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:49:00
4,2,7973,22480,2079,2,2249,N,46722127.0,81.0,173,...,7973.0,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:49:00


In [674]:
# Correct the day type

def tipo_dia(dia_anterior, tipo):
    if dia_anterior != 'S':
        return tipo
    else:
        return tipo - 1

st1['tipo_dia'] = np.vectorize(tipo_dia)(st1['dia_anterior'], st1['tipo_dia'])

In [675]:
# Create the trip_id

st1['trip_id'] =  st1['tipo_dia'].map(str) + '-' + st1['cod_variante'].map(str) + '-' + st1['frecuencia'].map(str)
st1.head()

,tipo_dia,cod_variante,frecuencia,cod_ubic_parada,ordinal,hora,dia_anterior,GID,COD_LINEA,DESC_LINEA,...,DESC_VARIA,COD_VAR_01,COD_ORIGEN,DESC_ORIGE,COD_DESTIN,DESC_DESTI,geometry,route_id,arrival_time,trip_id
0,1,7973,22520,2078,1,2253,N,46722127.0,81.0,173,...,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:53:00,1-7973-22520
1,1,7973,22520,2079,2,2253,N,46722127.0,81.0,173,...,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:53:00,1-7973-22520
2,1,7973,22520,2340,3,2255,N,46722127.0,81.0,173,...,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:55:00,1-7973-22520
3,2,7973,22480,2078,1,2249,N,46722127.0,81.0,173,...,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:49:00,2-7973-22480
4,2,7973,22480,2079,2,2249,N,46722127.0,81.0,173,...,B,7966.0,138.0,NUEVO CENTRO,58.0,LUIS A DE HERRERA Y GRAL FLORES,LINESTRING (-56.16889034436939 -34.86981536802...,972-PORTONES - NUEVO CENTRO,22:49:00,2-7973-22480


In [676]:
# Drop duplicate rows (trips)

st1 = st1.sort_values(['tipo_dia', 'cod_variante', 'frecuencia', 'ordinal'])
st1 = st1.drop_duplicates(subset=['tipo_dia', 'cod_variante', 'frecuencia', 'cod_ubic_parada', 'ordinal', 'hora',
                                  'GID', 'route_id', 'arrival_time', 'trip_id'], keep=False)

In [677]:
# Drop the trips that don't have a route_id
# We find that a small number of trips didn't have any match when joining
# with "routes". We can drop these without a big loss of data.

trips = st1[st1.route_id.notnull()]

In [678]:
# Group the information by trip 

trips = trips.groupby(['route_id', 'tipo_dia', 'trip_id', 'GID'], as_index = False).agg({'ordinal': 'min'})
trips = trips.iloc[:, 0:4]
trips.columns = ('route_id', 'service_id', 'trip_id', 'shape_id')
trips.head()

,route_id,service_id,trip_id,shape_id
0,1-CIUDAD VIEJA - MALVIN,1,1-1-10090,19462120.0
1,1-CIUDAD VIEJA - MALVIN,1,1-1-10270,19462120.0
2,1-CIUDAD VIEJA - MALVIN,1,1-1-10440,19462120.0
3,1-CIUDAD VIEJA - MALVIN,1,1-1-11000,19462120.0
4,1-CIUDAD VIEJA - MALVIN,1,1-1-11160,19462120.0


In [679]:
# Save the data frame
# Change the path to your output desired address

trips.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/trips.txt", index = False)

# Stop_times.txt

In [680]:
# Get the columns we care about
# Change the name of some of them
# Create the columns we're missing
# Filter only the values that have a trip_id that is in trips.txt

stop_times = st1[["trip_id","ordinal", "cod_ubic_parada", "arrival_time"]]
stop_times = stop_times.rename(index=str, columns={"cod_ubic_parada": 'stop_id', "ordinal": "stop_sequence"})
stop_times['departure_time'] = stop_times['arrival_time']
stop_times = stop_times[stop_times.trip_id.isin(trips.trip_id)]
stop_times.head()

,trip_id,stop_sequence,stop_id,arrival_time,departure_time
2205877,1-1-5300,1,4041,05:30:00,05:30:00
2205878,1-1-5300,2,4763,05:31:00,05:31:00
2205879,1-1-5300,3,4764,05:32:00,05:32:00
2205880,1-1-5300,4,4765,05:33:00,05:33:00
2205881,1-1-5300,5,4773,05:33:00,05:33:00


In [681]:
# Check if the stop_id has the same format in stop_times and stops

print(type(stop_times.stop_id[1]))
print(type(stops.stop_id[1]))

<class 'numpy.int64'>
<class 'str'>


In [682]:
# Drop for duplicated trips in stop_times

stop_times = stop_times.drop_duplicates(subset=['trip_id', 'stop_sequence'], keep=False)

In [683]:
# Match the data type for stop_id
# Merge stop_times and stops to check if there are any stops 
# that don't exist in stops.txt
stop_times['stop_id'] = stop_times.stop_id.astype(str)
stop_times = pd.merge(stop_times, stops, how='left', left_on=['stop_id'], right_on=['stop_id'])
stop_times.isnull().values.any()

True

In [684]:
# Drop the stops that are not in stops.txt

stop_times = stop_times.dropna(subset=['stop_name'])
stop_times.isnull().values.any()

False

In [685]:
# Keep only the columns that are interesting for us

stop_times = stop_times[['trip_id', 'stop_sequence', 'stop_id', 'arrival_time', 'departure_time']]
stop_times.head()

,trip_id,stop_sequence,stop_id,arrival_time,departure_time
0,1-1-5300,1,4041,05:30:00,05:30:00
1,1-1-5300,2,4763,05:31:00,05:31:00
2,1-1-5300,3,4764,05:32:00,05:32:00
3,1-1-5300,4,4765,05:33:00,05:33:00
4,1-1-5300,5,4773,05:33:00,05:33:00


In [649]:
# Save the data frame
# Change the path to your output desired address

stop_times.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/stop_times.txt", index = False)

# Shapes.txt

In [686]:
# Create a vector with length = number of shapes to iterate later

nshapes = list(range(0,len(rtrans.GID.unique())))
len(nshapes)

1220

In [687]:
# Create an empty DataFrame

shapes = pd.DataFrame(columns = ['shape_pt_lon', 'shape_pt_lat', 'shape_pt_sequence', 'shape_id'] )

In [688]:
# Fill the DataFrame with the points for each of the shapes

for n in nshapes:
    df = pd.DataFrame(data = list(rtrans.geometry.iloc[n].coords), columns = ('shape_pt_lon', 'shape_pt_lat')) 
    df['shape_pt_sequence'] = pd.DataFrame(data = list(range(1,len(df)+1)))
    df['shape_id'] = rtrans.GID[n]
    shapes = shapes.append(df)   

In [689]:
# Save the data frame
# Change the path to your output desired address

shapes.to_csv("/Users/santiagotoso/GoogleDrive/Master/Python/Montevideo/output/shapes.txt", index = False)